In [8]:
import os
import pickle
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier
import keras.backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from keras.regularizers import l1, l2, l1_l2
import seaborn as sns
import shap
from tensorflow.keras.metrics import RootMeanSquaredError
import sys
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, confusion_matrix, mean_squared_error
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
import pibd

In [9]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def custom_roc_auc_score(y_true, y_pred):
    if len(np.unique(y_true)) == 1:
        return 0.5  # Default value for single class case
    else:
        return roc_auc_score(y_true, y_pred)
auc_scorer = make_scorer(custom_roc_auc_score, greater_is_better=True, needs_proba=True)

rmse_scorer = make_scorer(rmse, greater_is_better=False,needs_proba=True)

In [ ]:
dataframe = pd.read_csv("data/future_responder.sample_class_for_studywise_crossvalidation.tsv",sep="\t", header=0)
dataset = dataframe.values
y=dataframe["future.response"]
y = LabelEncoder().fit_transform(y)
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
X=pd.read_csv("data/batch_effect.GLASV_otutab.ACHIKNS_ver.after_MMUPHin_disease_covar.asv_read_count.tsv",sep="\t",header=0,index_col=0)
X=np.log(X+1)
X=X.loc[dataframe["sample"],]
X=pointbiserialFilter(X,y)
X=combine_meta_with_data(X)
#X=X.set_index("sample")
dataframe=dataframe.set_index("sample")
groups=dataframe.loc[X.index,"study"]
groups=LabelEncoder().fit_transform(groups)
#X=X.reset_index()
X=X.to_numpy()

In [ ]:
param_grids = {
    'dnn': {
        'model__hidden_layers': range(1, 6),
        'model__units': [32, 64, 128, 256],
        'model__l1_value': [0.001, 0.01, 0.1],
        'model__l2_value': [0.001, 0.01, 0.1],
        'model__dropout_rate': [0.3, 0.5, 0.7],
        'fit__epochs': [50, 100, 150, 200],
        'fit__batch_size': [16, 32, 64, 128]
    },
    'log_reg': {
        'C': [0.01, 0.1, 1, 10, 100]
    },
    'knn': {
        'n_neighbors': range(1, 31),
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    },
    'dt': {
        'max_depth': range(1, 20),
        'min_samples_split': range(2, 20),
        'min_samples_leaf': range(1, 20)
    },
    'rf': {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_features': ['sqrt', 'log2', None],
        'max_depth': range(1, 20),
        'min_samples_split': range(2, 20),
        'min_samples_leaf': range(1, 20)
    },
    'gb': {
        'n_estimators': [100, 200, 300, 400, 500],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.5, 0.7, 1.0],
        'max_depth': range(1, 20)
    },

    'svc': {
        'C': [0.01, 0.1, 1, 10, 100],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'gamma': ['scale', 'auto']
    }
}

best_log_reg_score, best_log_reg_params = tune_model(LogisticRegression(), param_grids['log_reg'], X,y)
best_knn_score,best_knn_params = tune_model(KNeighborsClassifier(), param_grids['knn'], X, y)
best_dt_score,best_dt_params = tune_model(DecisionTreeClassifier(), param_grids['dt'], X, y)
best_rf_score,best_rf_params = tune_model(RandomForestClassifier(), param_grids['rf'], X, y)
best_gb_score,best_gb_params = tune_model(GradientBoostingClassifier(), param_grids['gb'], X, y)
best_svc_score,best_svc_params = tune_model(SVC(probability=True), param_grids['svc'], X, y)

print("log_reg")
print(best_log_reg_score)
print(best_log_reg_params)
print("knn")
print(best_knn_score)
print(best_knn_params)
print("dt")
print(best_dt_score)
print(best_dt_params)
print("rf")
print(best_rf_score)
print(best_rf_params)
print("gb")
print(best_gb_score)
print(best_gb_params)
print("svc")
print(best_svc_score)
print(best_svc_params)